## Importing the required data

- upload jobs residency xlsx table to 'data'

## Converting xlsx file to pandas dataframe

- install openpyxl, import load_workbook and pandas
- load xlsx workbook --> access required sheet
- convert to dataframe with .values

In [1]:
#to open xlsx files
!pip install openpyxl

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from openpyxl import load_workbook
wb2 = load_workbook('./data/Qualifications-by-economic-activity-status-borough.xlsx', data_only=True)

In [5]:
wb2.sheetnames

['Metadata',
 'Quals of Workless',
 'Quals of Unemployed Borough',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018']

In [152]:
ws2 = wb2["2014"]

In [153]:
df = pd.DataFrame(ws2.values)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,None,None,% of economically active with NVQ4+ - working age,None,None,None,% of economically active with NVQ4+ - working ...,None,None,None,...,None,None,% of economically active with no qualification...,None,None,None,% of economically active with no qualification...,None,None,None
1,Code,Area,number,denominator,percent,confidence,number,denominator,percent,confidence,...,percent,confidence,number,denominator,percent,confidence,number,denominator,percent,confidence
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,00AA,City of London,!,!,!,-,!,!,!,-,...,!,!,!,!,!,!,!,!,!,!
4,00AB,Barking and Dagenham,31100,89300,34.8,4.9,14100,48100,29.3,6.5,...,10.4,3.1,6500,48100,13.4,4.9,2800,41200,6.9,3.7


In [85]:
dfT = df.T

## Clearing dataframe

- drop last rows by slicing
- aggregate all value outside London into one 'Outside London' category with Code 'X00000000'
- make coloumn 0 and row 0 axes labels

In [86]:
number = dfT.loc[dfT[1]=="number"]

In [87]:
dfT.drop(number.index, axis=0, inplace=True)

In [88]:
denom = dfT.loc[dfT[1]=="denominator"]

In [89]:
dfT.drop(denom.index, axis=0, inplace=True)

In [90]:
conf = dfT.loc[dfT[1]=="confidence"]

In [91]:
dfT.drop(conf.index, axis=0, inplace=True)

In [92]:
male = dfT.iloc[3::3, :]

In [93]:
dfT.drop(male.index, axis=0, inplace=True)

In [94]:
female = dfT.iloc[3::2, :]

In [95]:
dfT.drop(female.index, axis=0, inplace=True)

In [96]:
dfT.drop(0, axis=1, inplace=True)
dfT.drop(2, axis=1, inplace=True)

In [97]:
dfT.iloc[2, 0] = 'NVQ4+'
dfT.iloc[3, 0] = 'NVQ3'
dfT.iloc[4, 0] = 'Trade Apprenticeship'
dfT.iloc[5, 0] = 'NVQ2'
dfT.iloc[6, 0] = 'NVQ1'
dfT.iloc[7, 0] = 'Other'
dfT.iloc[8, 0] = 'No Qualifications'

In [98]:
#bc. ! indicates that level close to 0
dfT.replace('!', 0, inplace=True)

In [99]:
df = dfT.T

In [100]:
df.columns = df.loc[1]
df.drop(1, axis=0, inplace=True)
df.head()

1,Code,Area,NVQ4+,NVQ3,Trade Apprenticeship,NVQ2,NVQ1,Other,No Qualifications
3,00AA,City of London,0,0,0,0,0,0,0
4,00AB,Barking and Dagenham,34.8,12,2.4,15.4,11.3,13.6,10.4
5,00AC,Barnet,49.9,14.6,1.3,8.7,7.4,11.6,6.4
6,00AD,Bexley,37.4,16.9,2.8,21.8,13.9,3.7,3.5
7,00AE,Brent,53.9,14.3,0.6,9.4,5.7,10.7,5.4


## Establishing an average qualification score
- assign all categories a certain ordinal value from 0 to 4.5:
    - NVQ4+: 4.5 bc. it includes all 4 and 5 level active persons
    - Trade Apprenticeship: 2.5 bc. such an apprenticeship can get assigned level 2 or 3 (https://www.gov.uk/government/publications/a-guide-to-apprenticeships)
    - Other: 2.25 bc that is the average btw. 0 and 4.5

In [101]:
df['Average score'] = (df['NVQ4+']*4.5 + df['NVQ3']*3 + df['Trade Apprenticeship']*2.5 + df['NVQ2']*2 + df['NVQ1']*1 + df['Other']*2.25 + df['No Qualifications']*0)/100

In [102]:
df

1,Code,Area,NVQ4+,NVQ3,Trade Apprenticeship,NVQ2,NVQ1,Other,No Qualifications,Average score
3,00AA,City of London,0,0,0,0,0,0,0,0
4,00AB,Barking and Dagenham,34.8,12,2.4,15.4,11.3,13.6,10.4,2.713
5,00AC,Barnet,49.9,14.6,1.3,8.7,7.4,11.6,6.4,3.225
6,00AD,Bexley,37.4,16.9,2.8,21.8,13.9,3.7,3.5,2.91825
7,00AE,Brent,53.9,14.3,0.6,9.4,5.7,10.7,5.4,3.35525
8,00AF,Bromley,50,16.8,2,13.7,9.9,5.7,1.9,3.30525
9,00AG,Camden,72.4,12.1,0.9,5.2,3.6,3.8,1.9,3.869
10,00AH,Croydon,45.6,18.9,4.9,12.8,6.9,7.6,3.4,3.2375
11,00AJ,Ealing,54.2,12.8,2.6,10.6,3.5,9.8,6.5,3.3555
12,00AK,Enfield,48.1,13.5,2.6,10.1,12.4,9.5,3.8,3.17425


In [69]:
PROC_DATA_PATH = '~/library/data/processed/'
df.to_csv(PROC_DATA_PATH + 'qualifications-average-2014.csv') 

## 2) Dataset with specific values over time 
- clear df so that only contains borough and no qualification
- import row No Qualifications of all worksheets and merge them 

### No qualifications over time

In [188]:
ws2 = wb2["2004"]

In [189]:
df04 = pd.DataFrame(ws2.values)
df04.tail()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
50,922,Northern Ireland,216800,759000,28.6,1.5,102600,423800,24.2,1.9,...,18.1,1.3,84100,423800,19.8,1.8,53200,335200,15.9,1.8
51,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
52,941,England and Wales,7566200,25875900,29.2,0.2,4014500,14056700,28.6,0.3,...,10.8,0.2,1514600,14056700,10.8,0.2,1277000,11819200,10.8,0.2
53,925,Great Britain,8427600,28398200,29.7,0.2,4443000,15387000,28.9,0.3,...,10.9,0.1,1652900,15387000,10.7,0.2,1430600,13011300,11,0.2
54,926,United Kingdom,8644400,29157200,29.6,0.2,4545600,15810800,28.7,0.3,...,11,0.1,1737000,15810800,11,0.2,1483800,13346400,11.1,0.2


In [190]:
nq04 = df04[76]

In [191]:
ws2 = wb2["2005"]
df05 = pd.DataFrame(ws2.values)
nq05 = df05[76]

In [192]:
ws2 = wb2["2006"]
df06 = pd.DataFrame(ws2.values)
nq06 = df06[76]

In [193]:
ws2 = wb2["2007"]
df07 = pd.DataFrame(ws2.values)
nq07 = df07[76]

In [194]:
ws2 = wb2["2008"]
df08 = pd.DataFrame(ws2.values)
nq08 = df08[76]

In [195]:
ws2 = wb2["2009"]
df09 = pd.DataFrame(ws2.values)
nq09 = df09[76]

In [196]:
ws2 = wb2["2010"]
df10 = pd.DataFrame(ws2.values)
nq10 = df10[76]

In [197]:
ws2 = wb2["2011"]
df11 = pd.DataFrame(ws2.values)
nq11 = df11[76]

In [198]:
ws2 = wb2["2012"]
df12 = pd.DataFrame(ws2.values)
nq12 = df12[76]

In [199]:
ws2 = wb2["2013"]
df13 = pd.DataFrame(ws2.values)
nq13 = df13[76]

In [200]:
ws2 = wb2["2014"]
df14 = pd.DataFrame(ws2.values)
nq14 = df14[76]

In [201]:
ws2 = wb2["2015"]
df15 = pd.DataFrame(ws2.values)
nq15 = df15[76]

In [202]:
ws2 = wb2["2016"]
df16 = pd.DataFrame(ws2.values)
nq16 = df16[76]

In [203]:
ws2 = wb2["2017"]
df17 = pd.DataFrame(ws2.values)
nq17 = df17[76]

In [204]:
ws2 = wb2["2018"]
df18 = pd.DataFrame(ws2.values)
nq18 = df18[76]

In [205]:
#using only the borough key of df
bulk = df.iloc[::, 2:]

In [206]:
df.drop(bulk, axis=1, inplace=True)

In [207]:
df = pd.concat([df, nq04], axis=1)
df = pd.concat([df, nq05], axis=1)
df = pd.concat([df, nq06], axis=1)
df = pd.concat([df, nq07], axis=1)
df = pd.concat([df, nq08], axis=1)
df = pd.concat([df, nq09], axis=1)
df = pd.concat([df, nq10], axis=1)
df = pd.concat([df, nq11], axis=1)
df = pd.concat([df, nq12], axis=1)
df = pd.concat([df, nq13], axis=1)
df = pd.concat([df, nq14], axis=1)
df = pd.concat([df, nq15], axis=1)
df = pd.concat([df, nq16], axis=1)
df = pd.concat([df, nq17], axis=1)
df = pd.concat([df, nq18], axis=1)
df

,Code,Area,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76
0,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent
2,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,00AA,City of London,!,!,!,!,!,!,!,!,!,!,!,!,!,!,!
4,00AB,Barking and Dagenham,16.3,16.4,17,18.3,17.5,12.7,12.9,8.6,8.2,11.4,10.4,9.3,11.5,10,7.8
5,00AC,Barnet,6.8,6.4,5.3,7.8,5.6,5.9,5.2,3.7,5,2.8,6.4,3.3,3.6,3.7,3
6,00AD,Bexley,8.6,8.5,6.3,11.2,8.2,6.9,4.3,4.8,5.6,2.7,3.5,7.7,3.4,3.1,3.5
7,00AE,Brent,11.6,8.2,7.2,6.5,5.1,6.8,2.1,8.2,8.3,5.7,5.4,4.6,3.4,5.1,4.8
8,00AF,Bromley,7.5,6.1,7.1,5.3,5.8,2.9,3.7,5.4,3.8,4,1.9,1.7,1.8,3,2.6
9,00AG,Camden,4.6,3.8,6.3,6.2,3.4,4.7,4,4,3.8,4.8,1.9,1.6,2.2,6.9,4


In [208]:
df.columns = ['Code', 'Area', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [209]:
df.head()

,Code,Area,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent
2,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,00AA,City of London,!,!,!,!,!,!,!,!,!,!,!,!,!,!,!
4,00AB,Barking and Dagenham,16.3,16.4,17,18.3,17.5,12.7,12.9,8.6,8.2,11.4,10.4,9.3,11.5,10,7.8


In [210]:
df.replace('!', 0, inplace=True)

In [211]:
df.drop(0, axis=0, inplace=True)
df.drop(1, axis=0, inplace=True)
df.drop(2, axis=0, inplace=True)
df.head()

,Code,Area,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,00AA,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00AB,Barking and Dagenham,16.3,16.4,17,18.3,17.5,12.7,12.9,8.6,8.2,11.4,10.4,9.3,11.5,10,7.8
5,00AC,Barnet,6.8,6.4,5.3,7.8,5.6,5.9,5.2,3.7,5,2.8,6.4,3.3,3.6,3.7,3
6,00AD,Bexley,8.6,8.5,6.3,11.2,8.2,6.9,4.3,4.8,5.6,2.7,3.5,7.7,3.4,3.1,3.5
7,00AE,Brent,11.6,8.2,7.2,6.5,5.1,6.8,2.1,8.2,8.3,5.7,5.4,4.6,3.4,5.1,4.8


In [212]:
PROC_DATA_PATH = '~/library/data/processed/'
df.to_csv(PROC_DATA_PATH + 'no-qualifications-overtime.csv') 

### NVQ4+ over time

In [229]:
ws2 = wb2["2004"]
df04 = pd.DataFrame(ws2.values)
nq04 = df04[4]

In [230]:
ws2 = wb2["2005"]
df05 = pd.DataFrame(ws2.values)
nq05 = df05[4]

In [231]:
ws2 = wb2["2006"]
df06 = pd.DataFrame(ws2.values)
nq06 = df06[4]

In [232]:
ws2 = wb2["2007"]
df07 = pd.DataFrame(ws2.values)
nq07 = df07[4]

In [233]:
ws2 = wb2["2008"]
df08 = pd.DataFrame(ws2.values)
nq08 = df08[4]

In [234]:
ws2 = wb2["2009"]
df09 = pd.DataFrame(ws2.values)
nq09 = df09[4]

In [235]:
ws2 = wb2["2010"]
df10 = pd.DataFrame(ws2.values)
nq10 = df10[4]

In [236]:
ws2 = wb2["2011"]
df11 = pd.DataFrame(ws2.values)
nq11 = df11[4]

In [237]:
ws2 = wb2["2012"]
df12 = pd.DataFrame(ws2.values)
nq12 = df12[4]

In [238]:
ws2 = wb2["2013"]
df13 = pd.DataFrame(ws2.values)
nq13 = df13[4]

In [239]:
ws2 = wb2["2014"]
df14 = pd.DataFrame(ws2.values)
nq14 = df14[4]

In [240]:
ws2 = wb2["2015"]
df15 = pd.DataFrame(ws2.values)
nq15 = df15[4]

In [241]:
ws2 = wb2["2016"]
df16 = pd.DataFrame(ws2.values)
nq16 = df16[4]

In [242]:
ws2 = wb2["2017"]
df17 = pd.DataFrame(ws2.values)
nq17 = df17[4]

In [243]:
ws2 = wb2["2018"]
df18 = pd.DataFrame(ws2.values)
nq18 = df18[4]

In [244]:
#using only the borough key of df
bulk = df.iloc[::, 2:]

In [245]:
df.drop(bulk, axis=1, inplace=True)

In [246]:
df = pd.concat([df, nq04], axis=1)
df = pd.concat([df, nq05], axis=1)
df = pd.concat([df, nq06], axis=1)
df = pd.concat([df, nq07], axis=1)
df = pd.concat([df, nq08], axis=1)
df = pd.concat([df, nq09], axis=1)
df = pd.concat([df, nq10], axis=1)
df = pd.concat([df, nq11], axis=1)
df = pd.concat([df, nq12], axis=1)
df = pd.concat([df, nq13], axis=1)
df = pd.concat([df, nq14], axis=1)
df = pd.concat([df, nq15], axis=1)
df = pd.concat([df, nq16], axis=1)
df = pd.concat([df, nq17], axis=1)
df = pd.concat([df, nq18], axis=1)
df

,Code,Area,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
0,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent
2,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,00AA,City of London,80.6,73.1,67.9,73.7,69.4,100,!,!,100,100,!,100,83.3,91.1,100
4,00AB,Barking and Dagenham,17.2,20.2,18.5,22.4,26,29.1,33.8,33.2,29.8,32.3,34.8,35.9,37.5,35,39.2
5,00AC,Barnet,41.6,46.9,42.3,47.6,48.6,49.5,53.2,57.3,53.9,56.8,49.9,55.4,65.2,62.2,61.5
6,00AD,Bexley,17.3,21.8,24.2,23.1,28.1,31.5,30.5,29.6,31.9,35.3,37.4,38.3,40.8,39.9,45.2
7,00AE,Brent,31.7,35.8,35,31.3,31.4,41,31,33.8,46.8,50.8,53.9,53.4,52.1,45.4,47.5
8,00AF,Bromley,38.4,33.6,39.1,39,36.4,39.3,41,47.1,49.7,51.9,50,51.5,54.2,50.1,53.5
9,00AG,Camden,55.7,60,60.1,59.1,59.8,62.6,63.5,69.4,69.6,69.8,72.4,73.5,72.2,71.2,74.5


In [247]:
df.columns = ['Code', 'Area', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [248]:
df.head()

,Code,Area,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent,percent
2,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,00AA,City of London,80.6,73.1,67.9,73.7,69.4,100,!,!,100,100,!,100,83.3,91.1,100
4,00AB,Barking and Dagenham,17.2,20.2,18.5,22.4,26,29.1,33.8,33.2,29.8,32.3,34.8,35.9,37.5,35,39.2


In [249]:
df.replace('!', 0, inplace=True)

In [250]:
df.drop(0, axis=0, inplace=True)
df.drop(1, axis=0, inplace=True)
df.drop(2, axis=0, inplace=True)
df.head()

,Code,Area,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,00AA,City of London,80.6,73.1,67.9,73.7,69.4,100,0,0,100,100,0,100,83.3,91.1,100
4,00AB,Barking and Dagenham,17.2,20.2,18.5,22.4,26,29.1,33.8,33.2,29.8,32.3,34.8,35.9,37.5,35,39.2
5,00AC,Barnet,41.6,46.9,42.3,47.6,48.6,49.5,53.2,57.3,53.9,56.8,49.9,55.4,65.2,62.2,61.5
6,00AD,Bexley,17.3,21.8,24.2,23.1,28.1,31.5,30.5,29.6,31.9,35.3,37.4,38.3,40.8,39.9,45.2
7,00AE,Brent,31.7,35.8,35,31.3,31.4,41,31,33.8,46.8,50.8,53.9,53.4,52.1,45.4,47.5


In [251]:
PROC_DATA_PATH = '~/library/data/processed/'
df.to_csv(PROC_DATA_PATH + 'nvq4+-qualifications-overtime.csv') 